# Theory

Let's call a given natal kick distribution model $M(\vec{\theta})$, where $\vec{\theta} = \{v_{ns}, \sigma_{ns}\}$ for the Mandel Muller 2020 model.

Any given pulsar data set from Deller can be expressed as a collection of equally likely data points $\{d_i\}$.

Then, the likelihood of reproducing the data set using our model is given by:
$$p(d_i | M) = \int p(d_i | v_i) \cdot p(v_i|M) \cdot d v_i$$

Using the data files from Deller, we can compute the posterior on the velocity distribution as: 
$$p(v_i | d_i) = \frac{p(d_i | v_i) \cdot \pi(v_i)}{p(d_i)}$$
where $\pi(v_i)$ is our prior on the velocity distribution, and $p(d_i)$ is a normalization.


We will assume a flat prior $\pi$, and we will ignore the normalization factor $p(d_i)$ since it doesn't affect the probability distribution, i.e. it is independent of $v_i$.

Therefore, we can make the simplifying assumption that 
$$p(v_i|d_i) = p(d_i|v_i).$$

We can now re-write the likelihood equation as:
$$p(d_i | M) \approx \int p(v_i | d_i) \cdot p(v_i|M) \cdot d v_i .$$

$p(v_i | d_i)$ can be read off from the posterior data, since it is simply the probability distribution of the velocity measurements.

Thus, the probability of drawing a given pulsar $d_i$ from a model $M$ is given by:
$$p(d_i|M) = \langle p(v_i | M) \rangle.$$

Here, $p(v_i|M)$ is the probability of drawing a given velocity, which appears in the data set, from model $M$. The average over all these probabilities gives the overall probability of drawing this pulsar from the model.

Finally, the probability of drawing all $N$ pulsars from model M is
$$p(d|M) = \prod_{i=1}^{N} p(d_i|M).$$

(RESOLVED) To get $p(v_i|M)$, for now we will simply assume that the model velocities are transverse. **In general, this is a poor assumption**. We must project all the velocities along isotropically distributed planar directions to get a modeled probability distribution for Transverse Velocity. I expect that the current calculation **underestimates** the true velocity multiplier.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from scipy.stats.kde import gaussian_kde
from numpy import linspace
import time
from scipy import interpolate
import os

In [2]:
from natal_kick_tools import mandel_muller_likelihood_functions as mmf

# Probability from COMPAS simulated models

In [3]:
# Define the models of interest
bh_kicks=[200]
# ns_kicks = [300, 350, 400, 450, 500, 550, 600, 650, 700]
ns_kicks=[400, 450, 550, 600, 650]
sigmas = [0.1, 0.2, 0.3, 0.4, 0.5]

# Define the location of the posterior data
pulsar_data_dir = "../correctedPulsarPosteriors/correctedVtData"

# Generate 2D Projected velocities

In [4]:
mmf.v3d_to_v2d(bh_kicks=bh_kicks, ns_kicks=ns_kicks, sigmas=sigmas, mode='bse', \
               work_dir = os.environ['WORK'] + f'/supernova_remnant_bse', output_dir='model_velocities_bse')

Loading Mandel Muller model data from /work2/08178/vkapil/frontera/supernova_remnant_bse/bh_200_ns_400_sigma_0.1_combined.h5
Loading Mandel Muller model data from /work2/08178/vkapil/frontera/supernova_remnant_bse/bh_200_ns_400_sigma_0.2_combined.h5
Loading Mandel Muller model data from /work2/08178/vkapil/frontera/supernova_remnant_bse/bh_200_ns_400_sigma_0.3_combined.h5
Loading Mandel Muller model data from /work2/08178/vkapil/frontera/supernova_remnant_bse/bh_200_ns_400_sigma_0.4_combined.h5
Loading Mandel Muller model data from /work2/08178/vkapil/frontera/supernova_remnant_bse/bh_200_ns_400_sigma_0.5_combined.h5
Loading Mandel Muller model data from /work2/08178/vkapil/frontera/supernova_remnant_bse/bh_200_ns_450_sigma_0.1_combined.h5
Loading Mandel Muller model data from /work2/08178/vkapil/frontera/supernova_remnant_bse/bh_200_ns_450_sigma_0.2_combined.h5
Loading Mandel Muller model data from /work2/08178/vkapil/frontera/supernova_remnant_bse/bh_200_ns_450_sigma_0.3_combined.h5


# Compute all the likelihoods for the 89 pulsars

In [5]:
start = time.time() 
p_models = mmf.get_pulsar_probability(pulsar_data_dir, model_data_dir='model_velocities_bse',\
                                      bh_kicks=bh_kicks, ns_kicks=ns_kicks, sigmas=sigmas,\
                                      output_dir='calculatedModelLikelihoods_bse')
end = time.time()

print("Complete calculation completed in:", end - start, "s")

Loading projected model data from model_velocities_bse/vns_400_sigma_0.1_velocities
Loading projected model data from model_velocities_bse/vns_400_sigma_0.2_velocities
Loading projected model data from model_velocities_bse/vns_400_sigma_0.3_velocities
Loading projected model data from model_velocities_bse/vns_400_sigma_0.4_velocities
Loading projected model data from model_velocities_bse/vns_400_sigma_0.5_velocities
Loading projected model data from model_velocities_bse/vns_450_sigma_0.1_velocities
Loading projected model data from model_velocities_bse/vns_450_sigma_0.2_velocities
Loading projected model data from model_velocities_bse/vns_450_sigma_0.3_velocities
Loading projected model data from model_velocities_bse/vns_450_sigma_0.4_velocities
Loading projected model data from model_velocities_bse/vns_450_sigma_0.5_velocities
Loading projected model data from model_velocities_bse/vns_550_sigma_0.1_velocities
Loading projected model data from model_velocities_bse/vns_550_sigma_0.2_vel

In [6]:
p_models = p_models/np.sum(p_models)
print(p_models)

[5.54491425e-05 2.23984211e-03 1.26487081e-02 1.57425538e-02
 4.47632271e-03 6.05361857e-04 1.73817189e-02 9.67498706e-02
 1.31940540e-01 3.01785019e-02 1.09045503e-03 2.22550880e-02
 1.75064199e-01 2.26402782e-01 5.26120319e-02 3.02567078e-04
 5.75912916e-03 6.35295518e-02 8.59275786e-02 2.07456470e-02
 4.39806983e-05 7.95990721e-04 9.36552231e-03 1.99913066e-02
 4.09530098e-03]
